In [1]:
import sys
import arcpy
from arcpy.sa import *
from arcpy.ia import *
import os

In [2]:
# nadpisywanie plików (domyślnie false)
arcpy.env.overwriteOutput=True

In [3]:
arcpy.CheckOutExtension("Spatial")
arcpy.CheckOutExtension("3D")

'CheckedOut'

In [4]:
arcpy.workspace = r'F:\studia\analizy\MyProject\MyProject.gdb'
arcpy.env.extent = r'F:\studia\analizy\MyProject\dane_kontrolne\rastry\nmt_lpis_plesna_1992.tif'
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS_1989_Poland_CS92")
arcpy.env.cellSize = 1

In [5]:
bubd_a = r'dane_kontrolne/bdot/BUBD_A.shp'
buffor = r'dane_kontrolne/bufor/bufor_adm_100.shp'
skdr_l = r'dane_kontrolne/bdot/SKDR_L.shp'
ptwp_a = r'dane_kontrolne/bdot/PTWP_A.shp'
swrs_l = r'dane_kontrolne/bdot/SWRS_L.shp'
ptlz_a = r'dane_kontrolne/bdot/PTLZ_A.shp'
gasPipeline = r'dane_kontrolne/gaz/gazociag.shp'
ground = r'dane_kontrolne/gleby/mapa_25000_Plesna.shp'
nmt = r'dane_kontrolne/rastry/nmt_lpis_plesna_1992.tif'
dzialki = r'dane_kontrolne/dzialki/dzialki.shp'

# Wydzielenie warstw z bdot10k

In [25]:
# hotele
hotels = arcpy.management.SelectLayerByAttribute(bubd_a, "NEW_SELECTION", "X_KOD LIKE 'BUBD05' OR X_KOD LIKE 'BUBD06'", None)
arcpy.conversion.ExportFeatures(hotels, "hotele")

<Result 'F:\\studia\\analizy\\MyProject\\MyProject.gdb\\hotele'>

In [35]:
# budynki mieszkalne
expression = "X_KOD LIKE 'BUBD01' OR X_KOD LIKE 'BUBD02' OR X_KOD LIKE 'BUBD03' OR X_KOD LIKE 'BUBD04'"
buildings = arcpy.management.SelectLayerByAttribute(bubd_a, "NEW_SELECTION", expression, None)
arcpy.conversion.ExportFeatures(buildings, "budynkiMieszkalne")

<Result 'F:\\studia\\analizy\\MyProject\\MyProject.gdb\\budynkiMieszkalne'>

In [36]:
# drogi
roads = arcpy.management.SelectLayerByAttribute(skdr_l, "NEW_SELECTION", "X_KOD LIKE 'SKDR%'", None)
arcpy.conversion.ExportFeatures(roads, "drogi")

<Result 'F:\\studia\\analizy\\MyProject\\MyProject.gdb\\drogi'>

In [37]:
# jeziora
lakes = arcpy.management.SelectLayerByAttribute(ptwp_a, "NEW_SELECTION", "X_KOD LIKE 'PTWP02' OR X_KOD LIKE 'PTWP03'", None)
arcpy.conversion.ExportFeatures(lakes, "jeziora")

<Result 'F:\\studia\\analizy\\MyProject\\MyProject.gdb\\jeziora'>

In [38]:
# rzeki
rivers = arcpy.management.SelectLayerByAttribute(swrs_l, "NEW_SELECTION", "X_KOD LIKE 'SWRS%'", None)
arcpy.conversion.ExportFeatures(rivers, "rzeki")

<Result 'F:\\studia\\analizy\\MyProject\\MyProject.gdb\\rzeki'>

In [39]:
# las
forest = arcpy.management.SelectLayerByAttribute(ptlz_a, "NEW_SELECTION", "X_KOD LIKE 'PTLZ01'", None)
arcpy.conversion.ExportFeatures(forest, "lasy")

<Result 'F:\\studia\\analizy\\MyProject\\MyProject.gdb\\lasy'>

# Definicje funkcji

In [31]:
def distCriterion(layer, mask, a, b, c=None, d=None, linear=None):
    distMap = EucDistance(layer)
    extractedDistMap = ExtractByMask(distMap, mask, 'INSIDE')
    if linear == "UP":
        """
             b---
            /
        ___a 
        """
        raster = FuzzyMembership(extractedDistMap, FuzzyLinear(a, b))
        return raster
    elif linear == "DOWN":
        """
        ---a
            \
             b___
        """
        raster = FuzzyMembership(extractedDistMap, FuzzyLinear(b, a))
        return raster
    else:
        """
             b---c
            /     \
        ___a       d___
        """
        rasterAB = FuzzyMembership(extractedDistMap, FuzzyLinear(a, b))
        rasterCD = FuzzyMembership(extractedDistMap, FuzzyLinear(d, c))
        raster = Merge([rasterAB, rasterCD], 'MIN')
        return raster

In [32]:
def slopeRaster(inRaster, mask):
    extractedRaster = ExtractByMask(inRaster, mask, 'INSIDE')
    slopeRaster = Slope(extractedRaster, "PERCENT_RISE")
    outRaster = FuzzyMembership(slopeRaster, FuzzyLinear(20, 15))
    return outRaster

In [33]:
def aspectRaster(inRaster, mask):
    extractedRaster = ExtractByMask(inRaster, mask, 'INSIDE')
    outAspect = Aspect(extractedRaster)
    rasterAB = FuzzyMembership(outAspect, FuzzyLinear(135, 150))
    rasterCD = FuzzyMembership(outAspect, FuzzyLinear(225, 210))
    outRaster = Merge([rasterAB, rasterCD], 'MIN')
    return outRaster

# Kryteria 1-9

In [34]:
# hotele
hotele_400 = distCriterion("hotele", buffor, 400, 600, linear="UP")
hotele_400.save('hotele_output')

In [40]:
# budynki mieszkalne
mieszkalne_25_50_125_150 = distCriterion("budynkiMieszkalne", buffor, 25, 50, 125, 150)
mieszkalne_25_50_125_150.save('budynkiMieszkalne_output')

In [41]:
# drogi
drogi_15_25_90_100 = distCriterion("drogi", buffor, 15, 25, 90, 100)
drogi_15_25_90_100.save('drogi_output')

In [42]:
# jeziora rozmyte
lakes_20_200_300 = distCriterion("jeziora", buffor, 20, 20.1, 200, 300)
lakes_20_200_300.save('jezioraRozmyte_output')

In [43]:
# jeziora ostre
lakes_20 = distCriterion("jeziora", buffor, 20, 20.1, linear="UP")
lakes_20.save('jezioraOstre_output')

In [44]:
# rzeki rozmyte
rivers_20_200_300 = distCriterion("rzeki", buffor, 20, 20.1, 200, 300)
rivers_20_200_300.save('rzekiRozmyte_output')

In [45]:
# rzeki ostre
rivers_20 = distCriterion("rzeki", buffor, 20, 20.1, linear="UP")
rivers_20.save('rzekiOstre_output')

In [46]:
# las
forest_raster = distCriterion("lasy", buffor, 0, 0.1, linear="UP")
forest_raster.save('lasy_output')

In [47]:
# gazociąg ostry
gasPipeline_25 = distCriterion(gasPipeline, buffor, 25, 25.1, linear="UP")
gasPipeline_25.save("gazociagOstry_output")

In [48]:
# gazociag rozmyty
gasPipeline_25_200 = distCriterion(gasPipeline, buffor, 25, 200, linear="UP")
gasPipeline_25_200.save("gazociagRozmyty_output")

In [49]:
# nachylenie
nmt_slope = slopeRaster(nmt, buffor)
nmt_slope.save('nmtSlope_output')

In [50]:
# nasłonecznienie
nmt_aspect = aspectRaster(nmt, buffor)
nmt_aspect.save('nmtAspect_output')

In [67]:
# gleba
groundQuery = "TYP NOT LIKE 'F' AND TYP NOT LIKE 'G' AND TYP NOT LIKE 'E' AND TYP NOT LIKE 'M' AND TYP NOT LIKE 'T' AND TYP NOT LIKE 'Fb' AND TYP NOT LIKE 'Fc' AND TYP NOT LIKE 'FG'"
newGround = arcpy.management.SelectLayerByAttribute(ground, "NEW_SELECTION", groundQuery, None)
ground_raster = distCriterion(newGround, buffor, 0, 0.1, linear="DOWN")
ground_raster.save("gleba_output")

In [54]:
# mapa kosztów
path = r'F:\studia\analizy\MyProject\dane_kontrolne/pokrycieTerenu'
layersList = os.listdir(path)
layers = []
for x in layersList:
    if x[-3:] == 'shp':
        layers.append(x[:-4])

pokrycie = arcpy.management.Merge(layers)

fieldname = "koszt"
expression = "setValue(!X_KOD!)"
codeblock = """
def setValue(kod):
    if kod in ['PTWP02','PTZB01','PTZB03','PTZB04','PTKM02']:
        return 200
    elif kod in ['PTNZ01','PTNZ02']:
        return 150
    elif kod in ['PTZB02','PTLZ01','PTUT03','PTKM01']:
        return 100
    elif kod == 'PTUT02':
        return 90
    elif kod in ['PTZB05','PTLZ02','PTLZ03','PTPL01']:
        return 50
    elif kod in ['PTUT04','PTUT05','PTUT01']:
        return 20
    elif kod in ['PTRK','PTRK02']:
        return 15
    elif kod in ['PTTR02','PTGN01','PTGN02','PTGN03','PTGN04']:
        return 1
    else:
        return 0
"""
arcpy.management.AddField(pokrycie, fieldname, "LONG")
arcpy.management.CalculateField(pokrycie, fieldname, expression, "PYTHON3", codeblock)
raster = arcpy.conversion.FeatureToRaster(pokrycie, "koszt")
extracted = ExtractByMask(raster, buffor, 'INSIDE')
extracted.save('mapa_kosztow')

# Łączenie

In [68]:
# rozmyte z wagami
rozmyteList1 = [['hotele_output', 'VALUE', 0.3], ['drogi_output', "VALUE", 0.2], ['budynkiMieszkalne_output', "VALUE", 0.1],
              ['jezioraRozmyte_output', "VALUE", 0.1], ['rzekiRozmyte_output', "VALUE", 0.1], ['nmtAspect_output', 'VALUE', 0.05],
              ['nmtSlope_output', "VALUE", 0.05], ['gazociagRozmyty_output', "VALUE", 0.1]]
rozmyte1 = WeightedSum(WSTable(rozmyteList1))
ostre1 = FuzzyOverlay(["lasy_output", "rzekiOstre_output", "jezioraOstre_output", "gazociagOstry_output", "gleba_output"], "AND")
wynik1 = arcpy.ia.RasterCalculator([rozmyte1, ostre1], ["x", "y"], "x*y")
wynik1.save('wynik_z_roznymi_wagami')

In [69]:
# rozmyte bez wag
rozmyteList2 = [['hotele_output', 'VALUE', 0.125], ['drogi_output', "VALUE", 0.125], ['budynkiMieszkalne_output', "VALUE", 0.125],
              ['jezioraRozmyte_output', "VALUE", 0.125], ['rzekiRozmyte_output', "VALUE", 0.125], 
              ['nmtAspect_output', 'VALUE', 0.125], ['nmtSlope_output', "VALUE", 0.125], ['gazociagRozmyty_output', "VALUE", 0.125]]
rozmyte2 = WeightedSum(WSTable(rozmyteList2))
ostre2 = FuzzyOverlay(["lasy_output", "rzekiOstre_output", "jezioraOstre_output", "gazociagOstry_output", "gleba_output"], "AND")
wynik2 = arcpy.ia.RasterCalculator([rozmyte2, ostre2], ["x", "y"], "x*y")
wynik2.save('wynik_z_rownymi_wagami')

In [70]:
# reklasyfikacja
reclass_rozne = arcpy.sa.Reclassify("wynik_z_roznymi_wagami", "VALUE", "0 0,500000 0;0,500000 1 1", "DATA")
reclass_rozne.save("reclass_rozne")
reclass_rowne = arcpy.sa.Reclassify("wynik_z_rownymi_wagami", "VALUE", "0 0,500000 0;0,500000 1 1", "DATA")
reclass_rowne.save("reclass_rowne")

In [73]:
# RÓŻNE WAGI
# raster to polygon
arcpy.conversion.RasterToPolygon("reclass_rozne", "wektor_rozne", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)
# select by gridcode
newTable_rozne = arcpy.management.SelectLayerByAttribute("wektor_rozne", "NEW_SELECTION", "gridcode = 1", None)
arcpy.analysis.TabulateIntersection("dzialki", "ID_DZIALKI", newTable_rozne, "tabela_dzialki_rozne", None, None, None, "UNKNOWN")
arcpy.management.AddJoin("dzialki", "ID_DZIALKI", "tabela_dzialki_rozne", "ID_DZIALKI", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
# select by %
percent_rozne = arcpy.management.SelectLayerByAttribute("dzialki", "NEW_SELECTION", "PERCENTAGE >= 70", None)
# dissolve
arcpy.management.Dissolve(percent_rozne, "dissolve_rozne", None, None, "SINGLE_PART", "DISSOLVE_LINES", '')
area_rozne = arcpy.management.SelectLayerByAttribute("dissolve_rozne", "NEW_SELECTION",
                                                     "Shape_area > 1000 AND Shape_area < 6000", None)
arcpy.management.CalculateField(area_rozne, "zwartosc", "(!Shape_Area!/(math.pi*(!Shape_Length!/(2*math.pi))**2))**0.5", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.conversion.ExportFeatures(area_rozne, "dzialki_pow_rozne")
arcpy.analysis.Select(area_rozne, "dzialka_koncowa_rozne", "zwartosc = (SELECT(MAX(zwartosc)) FROM dzialki_pow_rozne)")

<Result 'F:\\studia\\analizy\\MyProject\\MyProject.gdb\\dzialka_koncowa_rozne'>

In [74]:
# RÓWNE WAGI
# raster to polygon
arcpy.conversion.RasterToPolygon("reclass_rowne", "wektor_rowne", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)
# select by gridcode
newTable_rowne = arcpy.management.SelectLayerByAttribute("wektor_rowne", "NEW_SELECTION", "gridcode = 1", None)
arcpy.analysis.TabulateIntersection("dzialki", "ID_DZIALKI", newTable_rowne, "tabela_dzialki_rowne", None, None, None, "UNKNOWN")
arcpy.management.AddJoin("dzialki", "ID_DZIALKI", "tabela_dzialki_rowne", "ID_DZIALKI", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")
# select by %
percent_rowne = arcpy.management.SelectLayerByAttribute("dzialki", "NEW_SELECTION", "PERCENTAGE >= 70", None)
# dissolve
arcpy.management.Dissolve(percent_rowne, "dissolve_rowne", None, None, "SINGLE_PART", "DISSOLVE_LINES", '')
area_rowne = arcpy.management.SelectLayerByAttribute("dissolve_rowne", "NEW_SELECTION",
                                                     "Shape_area > 1000 AND Shape_area < 6000", None)
arcpy.management.CalculateField(area_rowne, "zwartosc", "(!Shape_Area!/(math.pi*(!Shape_Length!/(2*math.pi))**2))**0.5", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")
arcpy.conversion.ExportFeatures(area_rowne, "dzialki_pow_rowne")
arcpy.analysis.Select(area_rowne, "dzialka_koncowa_rowne", "zwartosc = (SELECT(MAX(zwartosc)) FROM dzialki_pow_rowne)")

<Result 'F:\\studia\\analizy\\MyProject\\MyProject.gdb\\dzialka_koncowa_rowne'>

# Przyłącze gazowe

In [7]:
# RÓŻNE WAGI
# mapa kosztów i kierunki
distance_raster_rozne = arcpy.sa.CostDistance("dzialka_koncowa_rozne", "mapa_kosztow", None, "kierunki_rozne", None, None, None, None, '')
distance_raster_rozne.save("mapa_kosztow_rozne")

In [77]:
# RÓŻNE WAGI
# najlepsze przyłącze
out_raster = arcpy.sa.CostPath(gasPipeline, "mapa_kosztow_rozne", "kierunki_rozne", "BEST_SINGLE", "Id", "INPUT_RANGE")
out_raster.save("przylacze_rozne_raster")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000865: Input cost distance raster: mapa_kosztow_rozne does not exist.
WARNING 001000: Destination field: Field Id does not exist
Failed to execute (CostPath).


In [78]:
# RÓŻNE WAGI
# raster to wektor
arcpy.conversion.RasterToPolyline("przylacze_rozne_raster", "przylacze_rozne_wektor", "ZERO", 0, "SIMPLIFY", "Value")

<Result 'F:\\studia\\analizy\\MyProject\\MyProject.gdb\\przylacze_rozne_wektor'>

In [78]:
# RÓWNE WAGI
# mapa kosztów i kierunki
distance_raster_rowne = arcpy.sa.CostDistance("dzialka_koncowa_rowne", "mapa_kosztow", None, "kierunki_rowne", None, None, None, None, '')
distance_raster_rowne.save("mapa_kosztow_rowne")
# najlepsze przyłącze
out_raster_rowne = arcpy.sa.CostPath(gasPipeline, distance_raster_rowne, "kierunki_rowne", "BEST_SINGLE", "Id", "INPUT_RANGE")
out_raster_rowne.save("przylacze_rowne_raster")
# raster to wektor
arcpy.conversion.RasterToPolyline("przylacze_rowne_raster", "przylacze_rowne_wektor", "ZERO", 0, "NO_SIMPLIFY", "Value")

ExecuteError: ERROR 010658: Failure in distributed raster analytics operation.
Failed to execute (CostDistance).
